# Clustering Crypto

In [69]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


## Deliverable 1: Preprocessing the Data for PCA

In [70]:
# Load the crypto_data.csv dataset.
file_to_load = "Resources/crypto_data.csv"
crypto_df = pd.read_csv(file_to_load, index_col=0)
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [71]:
crypto_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1252 entries, 42 to PUNK
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CoinName         1252 non-null   object 
 1   Algorithm        1252 non-null   object 
 2   IsTrading        1252 non-null   bool   
 3   ProofType        1252 non-null   object 
 4   TotalCoinsMined  744 non-null    float64
 5   TotalCoinSupply  1252 non-null   object 
dtypes: bool(1), float64(1), object(4)
memory usage: 59.9+ KB


#### Keep all the cryptocurrencies that are being traded.

In [72]:
# Search for null values in IsTrading column
crypto_df["IsTrading"].isnull().values.any()

False

In [73]:
# Search for cyrptocurrencies who are not being traded (isTrading is False)
# For the search, 1 = True and 0 = False

# Filter for isTrading = True and update the dataframe
crypto_df_processed = crypto_df[crypto_df["IsTrading"] == 1]
crypto_df_processed.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


#### Keep all the cryptocurrencies that have a working algorithm.

In [74]:
# Keep all the cryptocurrencies that have a working algorithm.
# Checking to see if any Algorithm values are empty
crypto_df_processed["Algorithm"].isnull().values.any()

False

#### Remove the "IsTrading" column. 

In [75]:
# Remove the "IsTrading" column. 
crypto_df_processed = crypto_df_processed.drop(columns=['IsTrading'])
crypto_df_processed.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


#### Remove rows that have at least 1 null value.

In [76]:
crypto_df_processed = crypto_df_processed.dropna()
crypto_df_processed.head()
# len(crypto_df_processed)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


#### Keep the rows where coins are mined.

In [77]:
# Keep the rows where coins are mined.
crypto_df_processed = crypto_df_processed[crypto_df_processed['TotalCoinsMined'] > 0]
crypto_df_processed.head()
# len(crypto_df_processed)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


#### Create a new DataFrame that holds only the cryptocurrencies names.

In [78]:
crypto_names_df = crypto_df_processed.filter(["CoinName"], axis=1)
crypto_names_df.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


#### Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.

In [79]:
crypto_df_processed = crypto_df_processed.drop(columns=['CoinName'])
crypto_df_processed.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
LTC,Scrypt,PoW,6.303924e+07,84000000
DASH,X11,PoW/PoS,9.031294e+06,22000000
XMR,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethash,PoW,1.133597e+08,210000000
ZEC,Equihash,PoW,7.383056e+06,21000000


#### Use ```get_dummies()``` to create variables for text features.

In [80]:
crypto_df_processed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 532 entries, 42 to XBC
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Algorithm        532 non-null    object 
 1   ProofType        532 non-null    object 
 2   TotalCoinsMined  532 non-null    float64
 3   TotalCoinSupply  532 non-null    object 
dtypes: float64(1), object(3)
memory usage: 20.8+ KB


In [81]:
crypto_df_processed.dtypes[crypto_df_processed.dtypes == object]

Algorithm          object
ProofType          object
TotalCoinSupply    object
dtype: object

In [82]:
X = pd.get_dummies(crypto_df_processed, columns=["Algorithm", "ProofType"])
#X = X.drop(columns=['TotalCoinsMined', 'TotalCoinSupply'])
X.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [83]:
# Standardize the data with StandardScaler().
X_scaled = StandardScaler().fit_transform(X)
print(X_scaled[0:1])

[[-0.11710817 -0.1528703  -0.0433963  -0.0433963  -0.0433963  -0.06142951
  -0.07530656 -0.0433963  -0.06142951 -0.06142951 -0.0433963  -0.0433963
  -0.19245009 -0.06142951 -0.09740465 -0.0433963  -0.11547005 -0.07530656
  -0.0433963  -0.0433963  -0.15191091 -0.0433963  -0.13118084 -0.0433963
  -0.0433963  -0.08703883 -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.06142951 -0.0433963  -0.08703883 -0.08703883 -0.08703883 -0.0433963
  -0.13118084 -0.13840913 -0.13840913 -0.0433963  -0.06142951 -0.0433963
  -0.07530656 -0.18168574 -0.0433963  -0.0433963  -0.0433963  -0.07530656
  -0.15826614 -0.31491833 -0.0433963  -0.08703883 -0.07530656 -0.06142951
   1.38675049 -0.0433963  -0.0433963  -0.06142951 -0.0433963  -0.0433963
  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.39879994 -0.0433963  -0.18168574 -0.0433963  -0.08703883 -0.08703883
  -0.10680283 -0.0433963  -0.13118084 -0.0433963  -0.0433963  -0.0433963
  -0.0433963  -0.07530656 -0.43911856 -0.04339

## Deliverable 2: Reducing Data Dimensions Using PCA

#### Using PCA to reduce dimension to three principal components.

In [84]:
pca = PCA(n_components=3)

In [85]:
X_pca = pca.fit_transform(X_scaled)

#### Create a DataFrame with the three principal components.

In [86]:
pcs_df = pd.DataFrame(
    data = X_pca, 
    columns=["PC 1", "PC 2", "PC 3"],
    index=X.index
)
pcs_df.head(10)

,PC 1,PC 2,PC 3
42,-0.339342,1.029222,-0.470523
404,-0.322671,1.029347,-0.470988
1337,2.310750,1.636274,-0.596506
BTC,-0.143242,-1.338201,0.061909
ETH,-0.154425,-1.991964,0.287704
LTC,-0.175137,-1.123116,0.027094
DASH,-0.380225,1.240134,-0.480332
XMR,-0.158510,-2.178622,0.447736
ETC,-0.152866,-1.992069,0.287684
ZEC,-0.143365,-1.920999,0.347338


## Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [87]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")


E:\anaconda3\envs\PythonData\lib\site-packages\sklearn\cluster\_kmeans.py:1040: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [88]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)
# predictions

In [89]:
# pcs_df.head()

In [90]:
crypto_df.head()
crypto_df = crypto_df.drop(columns=['CoinName'])

In [91]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
# frames = [crypto_df, pcs_df, crypto_names_df]
#frames = [crypto_df, pcs_df, crypto_names_df]
#clustered_df = pd.concat(frames, axis=1)
#clustered_df = clustered_df.dropna()
#clustered_df = clustered_df.drop(columns=['IsTrading'])
# clustered_df.head(10)

In [92]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = crypto_df.join(pcs_df, how='inner')
clustered_df = clustered_df.join(crypto_names_df, how='inner')
clustered_df.head(10)

,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName
42,Scrypt,True,PoW/PoS,4.199995e+01,42,-0.339342,1.029222,-0.470523,42 Coin
404,Scrypt,True,PoW/PoS,1.055185e+09,532000000,-0.322671,1.029347,-0.470988,404Coin
1337,X13,True,PoW/PoS,2.927942e+10,314159265359,2.310750,1.636274,-0.596506,EliteCoin
BTC,SHA-256,True,PoW,1.792718e+07,21000000,-0.143242,-1.338201,0.061909,Bitcoin
ETH,Ethash,True,PoW,1.076842e+08,0,-0.154425,-1.991964,0.287704,Ethereum
LTC,Scrypt,True,PoW,6.303924e+07,84000000,-0.175137,-1.123116,0.027094,Litecoin
DASH,X11,True,PoW/PoS,9.031294e+06,22000000,-0.380225,1.240134,-0.480332,Dash
XMR,CryptoNight-V7,True,PoW,1.720114e+07,0,-0.158510,-2.178622,0.447736,Monero
ETC,Ethash,True,PoW,1.133597e+08,210000000,-0.152866,-1.992069,0.287684,Ethereum Classic
ZEC,Equihash,True,PoW,7.383056e+06,21000000,-0.143365,-1.920999,0.347338,ZCash


In [93]:
clustered_df = clustered_df.drop(columns=['IsTrading'])

In [94]:
#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df['class'] = model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.339342,1.029222,-0.470523,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.322671,1.029347,-0.470988,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.310750,1.636274,-0.596506,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.143242,-1.338201,0.061909,Bitcoin,1
ETH,Ethash,PoW,1.076842e+08,0,-0.154425,-1.991964,0.287704,Ethereum,1
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.175137,-1.123116,0.027094,Litecoin,1
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.380225,1.240134,-0.480332,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.158510,-2.178622,0.447736,Monero,1
ETC,Ethash,PoW,1.133597e+08,210000000,-0.152866,-1.992069,0.287684,Ethereum Classic,1
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.143365,-1.920999,0.347338,ZCash,1


## Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [95]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="class",
    symbol="class",
    width=800,
    hover_name="CoinName",
    hover_data=["CoinName", "Algorithm"],
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()


In [96]:
# Create a table with tradable cryptocurrencies.
clustered_df_table = clustered_df.hvplot.table(columns=['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'class'], sortable=True, selectable=True)
clustered_df_table

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,class]

In [97]:
# Print the total number of tradable cryptocurrencies.
print(f"The total number of tradable cryptocurrencies is:", len(clustered_df))

The total number of tradable cryptocurrencies is: 532


In [98]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
to_scale = clustered_df.copy()
to_scale = to_scale.drop(columns=['Algorithm', 'ProofType', 'PC 1', 'PC 2', 'PC 3', 'CoinName', 'class'])
#to_scale.head()
X_scaled = MinMaxScaler().fit_transform(to_scale)
print(X_scaled)

[[0.00000000e+00 4.20000000e-11]
 [1.06585544e-03 5.32000000e-04]
 [2.95755135e-02 3.14159265e-01]
 ...
 [9.90135079e-04 1.40022261e-03]
 [7.37028150e-06 2.10000000e-05]
 [1.29582282e-07 1.00000000e-06]]


In [99]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df = pd.DataFrame(data=X_scaled, index=clustered_df.index, columns=['TotalCoinsMined', 'TotalCoinSupply'])
plot_df = plot_df.iloc[:,[1,0]]
plot_df.head()

,TotalCoinSupply,TotalCoinsMined
42,4.200000e-11,0.000000
404,5.320000e-04,0.001066
1337,3.141593e-01,0.029576
BTC,2.100000e-05,0.000018
ETH,0.000000e+00,0.000109


In [100]:
# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df = plot_df.join(clustered_df["CoinName"])
plot_df.head()

,TotalCoinSupply,TotalCoinsMined,CoinName
42,4.200000e-11,0.000000,42 Coin
404,5.320000e-04,0.001066,404Coin
1337,3.141593e-01,0.029576,EliteCoin
BTC,2.100000e-05,0.000018,Bitcoin
ETH,0.000000e+00,0.000109,Ethereum


In [101]:
# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df = plot_df.join(clustered_df["class"])
plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,class
42,4.200000e-11,0.000000,42 Coin,0
404,5.320000e-04,0.001066,404Coin,0
1337,3.141593e-01,0.029576,EliteCoin,0
BTC,2.100000e-05,0.000018,Bitcoin,1
ETH,0.000000e+00,0.000109,Ethereum,1
LTC,8.400000e-05,0.000064,Litecoin,1
DASH,2.200000e-05,0.000009,Dash,0
XMR,0.000000e+00,0.000017,Monero,1
ETC,2.100000e-04,0.000115,Ethereum Classic,1
ZEC,2.100000e-05,0.000007,ZCash,1


In [102]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(x="TotalCoinsMined", y="TotalCoinSupply", by="class", hover_cols=['CoinName'])


:NdOverlay   [class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)